In [24]:
import csv
import numpy as np
import cv2
from time import time
import os
from PIL import Image as pilimg

In [25]:
person_labels=['/m/04yx4', '/m/03bt1vf', '/m/01bl7v', '/m/05r655', '/m/01g317'] #merger for 'person'
bag_labels=['/m/01940j'] #merger for 'bag' '/m/0hf58v5'

In [26]:
root_dir='/mnt/openimgv4/'
#lbl_dir=root_dir+'tools/train-annotations-bbox_people_and_bag.csv'
lbl_dir=root_dir+'tools/validation-annotations-bbox_people_and_bag.csv'
cls_dir_num=root_dir+'clsDict_new.csv'
cls_dir_code=root_dir+'train/label/challenge-2018-class-descriptions-500.csv'
#dat_dir=root_dir+'sources/train_img/'
dat_dir=root_dir+'sources/valid_img/'
rec_dir=root_dir+'sources/rec_files/'

#prefix='train_bag'
prefix='valid_bag'

In [27]:
# convert csv file to dict
def row_csv2dict(csv_file):
    dict_club={}
    with open(csv_file)as f:
        reader=csv.reader(f,delimiter=',')
        for row in reader:
            dict_club[row[0]]=row[1]
    return dict_club

In [28]:
# search for class name for a given label
def get_cls(lbl):
    cls_dict=row_csv2dict(cls_dir_code)
    if lbl in cls_dict:
        return cls_dict[lbl]
    else:
        return 'None' # return 'None' for empty labels

In [29]:
# rotate vertical images
def rotatecc(id):
    im = pilimg.open(dat_dir+id+'.jpg')
    ng = im.transpose(pilimg.ROTATE_90)
    ng.save(dat_dir+id+'_r.jpg')
    with open(root_dir+'sources/valid_rotated_cc.log', 'a+') as f:
        f.write(id+'\n')

In [30]:
# search for labels for a given image id
id_idx=0 #index of image id
lbl_idx=[2, 4, 5, 6, 7] #label_name, xmin, xmax, ymin, ymax

# only preserve non-group labels
gp_idx=10

start=time()
with open(lbl_dir,encoding='utf-8') as cfile:
    reader = csv.reader(cfile)
    readeritem=[]
    readeritem.extend([row for row in reader])
with open(rec_dir+prefix+'.lst', 'w+') as f:
    isgot=0
    id_old='666'
    img_idx=0
    for i,rows in enumerate(readeritem):
        if i%1000==0:
            print(i,' time:',time()-start)
        if rows[2] in bag_labels:
            path = dat_dir + rows[id_idx] + '.jpg'  
            if os.path.isfile(path):
                if True:
                    if rows[id_idx]!=id_old:
                        if id_old!='666':
                            if box_count>0:
                                f.write(
                                str(img_idx) + '\t' +
                                str(4) + '\t' + str(5) + '\t' +
                                str(W) + '\t' + str(H) + '\t' +
                                mcstr +
                                str(file_id) + '.jpg\n'
                                )
                                img_idx+=1
                        mcstr=''
                        box_count=0
                        img=cv2.imread(path)
                        W=np.size(img,1)
                        H=np.size(img,0)
                        row=rows
                        id_old=rows[id_idx]
                        if H>W:
                            file_id=rows[id_idx]+'_r'
                            rotatecc(rows[id_idx])
                            tmp=W
                            W=H
                            H=tmp
                            isRotated=True
                        else:
                            file_id=rows[id_idx]
                            isRotated=False

                    if rows[2] in bag_labels:
                        mcstr=mcstr+str(0)+'\t' # class 0
                        if isRotated==False:
                            mcstr+=str(rows[4]) + '\t' + str(rows[6]) + '\t' 
                            mcstr+=str(rows[5]) + '\t' + str(rows[7]) + '\t'
                        else:
                            mcstr+=str(rows[6]) + '\t' + str(1-float(rows[5])) + '\t' 
                            mcstr+=str(rows[7]) + '\t' + str(1-float(rows[4])) + '\t'
                        box_count+=1

0  time: 1.2668161392211914
1000  time: 1.2673790454864502
2000  time: 1.3046000003814697
3000  time: 1.3342206478118896
4000  time: 1.441697359085083
5000  time: 1.4875636100769043
6000  time: 1.487973928451538
7000  time: 1.4883418083190918
8000  time: 1.4887125492095947
9000  time: 1.498152494430542
10000  time: 1.5103116035461426
11000  time: 1.5584468841552734
12000  time: 1.7308130264282227
13000  time: 1.731231451034546
14000  time: 1.8119566440582275
15000  time: 1.8731870651245117
16000  time: 1.8735923767089844
17000  time: 1.873948574066162
18000  time: 1.8742973804473877
19000  time: 1.8859381675720215
20000  time: 1.8864169120788574
21000  time: 1.8868577480316162
22000  time: 1.99623441696167
